# Jupyter Class Helper
---
These files are used to configure and organize the website's contents.

In [1]:
#%load_ext autoreload
#%autoreload 2
#%matplotlib inline


In [2]:
# Always run this before any of the following cells
import pandas as pd
import numpy as np
import csv
import logging
import subprocess
import yaml
import builder as bd
from pathlib import Path
base_path=Path('..')
config_path = base_path / 'config'
cf=bd.load_yaml_file(config_path / 'config.yml')
excel_file= config_path / cf['excel_file']
class_path= base_path / cf['class']
content_path = class_path / 'content'



In [3]:
# These load configuration from the excel files 
config = bd.load_yaml_file(class_path / '_config.yml') # Load the file.
toc = bd.load_yaml_file(class_path / '_toc.yml') # Load the file.
config_xl= pd.read_excel(excel_file, sheet_name = '_config_yml', header=None, index_col=None)
schedule= pd.read_excel(excel_file, sheet_name = 'Schedule',  index_col=None)
content={}
content['Before Class']= pd.read_excel(excel_file, sheet_name = 'Before',  index_col=None)
content['In Class']= pd.read_excel(excel_file, sheet_name = 'During',  index_col=None)
content['Assignment']= pd.read_excel(excel_file, sheet_name = 'Assignments',  index_col=None)


In [4]:
#Create the syllabus link.
#The second value of the index postion of the syllabus on the before class content.
#bd.create_syllabus(content['Before Class'],0,cf['syllabus_message'],content_path / 'syllabus.md', config['repository']['url'])

In [5]:
#Fix in case individual tries to publish where session is NA. This isn't allowed. 
schedule.loc[schedule['Session'].isna(),'Publish']=0. 

In [6]:
#Generate Links from the schedule to the sessions and within the other tables. 
schedule.loc[schedule['Publish']==1,'Location']=schedule.loc[schedule['Publish']==1,'Session'].apply(lambda x: '../sessions/session'+str(int(x)))
schedule.loc[schedule['Publish']==1,'Type']='Markdown'

In [7]:
schedule

,Week,Session,Date,Day,Topic,Summary,Publish,Location,Type
0,1,1,2020-09-02,Wed,"Brent Solina '12 (Founder and CTO, MICROrganic...","As a biochem/biophysics major at RPI, Brent So...",1.0,../sessions/session1,Markdown
1,2,2,2020-09-09,Wed,"John Haller '86 (VP Technology, Sports Illustr...",NaN,1.0,../sessions/session2,Markdown
2,3,3,2020-09-16,Wed,William Fosnight '92 (Co-Founder and Chief Dev...,Where does an engineer go after holding the hi...,1.0,../sessions/session3,Markdown
3,4,4,2020-09-23,Wed,"Dr. Patrice Milos-Spalding '82 (Co-Founder, Pr...",Doctors diagnose diseases and treat patients d...,1.0,../sessions/session4,Markdown
4,5,5,2020-09-30,Wed,"Dwaine Alleyne '04 (Co-Founder and Partner, En...",When a scandal within the industry you are bui...,1.0,../sessions/session5,Markdown
5,6,6,2020-10-07,Wed,"Jeff Stewart '91 (Chairman and Co-Founder, Len...",What happens when the platform needed to make ...,1.0,../sessions/session6,Markdown
6,7,7,2020-10-14,Wed,"Eben Bayer '07 (Co-Founder and CEO, Ecovative ...",When you’re college professor always said “don...,1.0,../sessions/session7,Markdown
7,8,8,2020-10-21,Wed,"Tobi Saulnier '84 (Founder and CEO, 1st Playab...",With 3 degrees in electrical engineering from ...,1.0,../sessions/session8,Markdown
8,9,9,2020-10-28,Wed,"Vikram Agrawal (President & CEO, Levrx Technol...","President & CEO, Levrx Technology; Co-CEO, Agr...",1.0,../sessions/session9,Markdown
9,10,10,2020-11-04,Wed,TBD,TBD,0.0,NaN,NaN


In [8]:
config['repository']['url']

'https://github.com/SeverinoCenter/impactful_innovators_website'

In [9]:
#schedule=bd.link_generator(schedule, 'Summary',config['repository']['url'],'Link')

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [10]:

#schedule=bd.link_generator(schedule, 'Summary',config['repository']['url'],'Link')
content['Assignment']=bd.link_generator(content['Assignment'], 'Assignment',config['repository']['url'],'Starter')
content['Before Class']=bd.link_generator(content['Before Class'], 'Content',config['repository']['url'],'Link')
content['In Class']=bd.link_generator(content['In Class'], 'Content',config['repository']['url'],'Link')

In [11]:
#Get the in class activities and prepare and output a markdown file. 
schedule_ic=schedule.merge(content['In Class'], left_on='Session', right_on='Session', how='left')
schedule_ic= schedule_ic.loc[schedule_ic['Content'].notnull(),['Week', 'Session', 'Date', 'Content']]
schedule_ic=bd.pandas_to_md(schedule_ic, content_path / 'in_class.md', 'In Class', \
        include = ['Week', 'Session', 'Date', 'Content'], header=cf['in_class_header'])

Converting datetime to 
Outputting file: ../site/content/in_class.md


In [12]:
#Get the before class activities and prepare and output a markdown file. 
schedule_bc=schedule.merge(content['Before Class'], left_on='Session', right_on='Session', how='left')
schedule_bc= schedule_bc.loc[schedule_bc['Content'].notnull(),['Week', 'Session', 'Date', 'Content']]
schedule_bc=bd.pandas_to_md(schedule_bc, content_path / 'preparation.md', 'Before Class', \
                             include = ['Week', 'Session', 'Date', 'Content'], header=cf['bc_class_header'])
schedule=schedule.merge(content['Assignment'], left_on='Session', right_on='Session', how='left')



Converting datetime to 
Outputting file: ../site/content/preparation.md


In [13]:
#Get the assignments and prepare and output a markdown file. 
assignments_new = schedule.loc[schedule['Assignment'].notnull(),['Week', 'Session', 'Date', 'Assignment', 'Due']]
assignments_new=bd.pandas_to_md(assignments_new, content_path / 'assignments.md', 'Assignments', \
                             include = ['Week', 'Session', 'Date', 'Assignment', 'Due'],header=cf['assignments_header'])

Converting datetime to 
Converting datetime to 
Outputting file: ../site/content/assignments.md


In [14]:
#Output the schedule to markdown.
schedule=bd.pandas_to_md(schedule, content_path / 'schedule.md', 'Schedule', \
                             include = ['Week', 'Session', 'Date', 'Day', 'Topic', 'Summary', 'Assignment', 'Due'],header=cf['schedule_header'])

Converting datetime to 
Converting datetime to 
Outputting file: ../site/content/schedule.md


In [15]:
#Generate Session Files
toc=bd.generate_sessions(config, toc, 2, schedule, class_path / 'sessions',content, ['Before Class', 'In Class', 'Assignment'])

Outputting  session1.md
Outputting  session2.md
Outputting  session3.md
Outputting  session4.md
Outputting  session5.md
Outputting  session6.md
Outputting  session7.md
Outputting  session8.md
Outputting  session9.md


In [16]:
#Update the sessions to the yaml file.  Other updates to notebooks need to be done manually.
bd.update_yaml_file(class_path / '_toc.yml', toc)

Updating the file: ../site/_toc.yml


In [17]:
#TBD Make it so that notebooks will show up in _toc.yml. 